In [ ]:
!pip list

In [ ]:
!pip -q install torch==2.0.0 git+https://github.com/huggingface/diffusers transformers accelerate imageio[ffmpeg]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


**Need to mount your google drive to save output video. The output .mp4 is not viewable within google colab.**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# customize the output name
vid_name_1 = 'before_upscale'

Import Libraries

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from PIL import Image

pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)

# optimize for GPU memory
pipe.enable_model_cpu_offload()






text_encoder/model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Memory optimization

In [ ]:
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1)
pipe.enable_vae_slicing()

prompt = " Rabbit is eating a carrot"
video_duration_seconds = 3 #@param {type:"integer"}
num_frames = video_duration_seconds * 8
output_name=f'/content/{vid_name_1}.mp4'
video_frames = pipe(prompt, num_frames=num_frames).frames
video_path = export_to_video(video_frames,output_video_path=output_name)
video_path

  0%|          | 0/50 [00:00<?, ?it/s]

'/content/before_upscale.mp4'

Display the video

In [ ]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

def display_video(video):
    fig = plt.figure(figsize=(4.2,4.2))  #Display size specification
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=100, repeat_delay=1000)

    plt.close()
    return anime
video = imageio.mimread(video_path)  #Loading video
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5

Clean the GPU

In [ ]:
torch.cuda.empty_cache()

Customize output video name

In [ ]:
# customize the output name after the upscaling
vid_name_2 = 'after_upscale'

Upscale the video.

In [7]:
# load pipeline
pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_XL", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# memory optimization

pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1)
pipe.enable_vae_slicing()


# generate
video = [Image.fromarray(frame).resize((1024, 576)) for frame in video_frames]
output_name=f'/content/{vid_name_2}.mp4'
video_frames = pipe(prompt, video=video, strength=0.66).frames
video_path = export_to_video(video_frames,output_video_path=output_name)
print(video_path)

text_encoder/model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

/content/after_upscale.mp4


Restart Run time

In [ ]:
import os

# Restart the Colab runtime
os.kill(os.getpid(), 9)
